## Logistic Regression

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

#### defines variables

In [2]:
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")

#### Reads from csv file Titanic

In [3]:
data = pd.read_csv('data/Titanic.csv')
data["Age"].fillna(0, inplace=True)
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### gets inputs

In [4]:
def inputs():
    data.replace({"Sex": {"male": 1, "female": 0}}, inplace=True)
    # data = pd.get_dummies(data, columns=['Pclass'])
    train_data = data.sample(frac=0.8, random_state=99)
    test_data = data.loc[~data.index.isin(train_data.index), :]
    X_train = train_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_train = pd.get_dummies(X_train, columns=['Pclass'])
    X_test = test_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_test = pd.get_dummies(X_test, columns=['Pclass'])
    Y_train = train_data.loc[:, ['Survived']]
    Y_test = test_data.loc[:, ['Survived']]
    return X_train, Y_train, X_test, Y_test


X_train, Y_train, X_test, Y_test = inputs()

In [5]:
X_train.head()

,Sex,Age,Pclass_1,Pclass_2,Pclass_3
247,0,24.0,0,1,0
823,0,27.0,0,0,1
566,1,19.0,0,0,1
666,1,25.0,0,1,0
199,0,24.0,0,1,0


In [6]:
Y_train.head()

,Survived
247,1
823,1
566,0
666,0
199,0


#### Inference

In [7]:
def inference(X):
    return tf.sigmoid(tf.matmul(tf.cast(X, tf.float32), W) + b)

#### Loss function using Cross-Entropy

In [8]:
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        logits=tf.matmul(tf.cast(X, tf.float32), W) + b,
        labels=tf.cast(Y, tf.float32)))

In [9]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

#### Train based on batch size

In [10]:
def mini_batch(size):
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        
        training_steps = 100
        for epoch in range(training_steps):
            for i in range(0, X_train.shape[0], size):
                session.run(train_op, feed_dict={x: X_train[i:i+size], y: Y_train[i:i+size]})
    
        evaluate(session, X_test, Y_test)
        session.close()

#### Evaluating chance of survival ( greater than 0.5 will be choose as survived )

In [11]:
def evaluate(session, X, Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print("Accuracy : ")
    print(session.run(tf.reduce_mean(tf.cast(tf.equal(predicted, tf.cast(Y, tf.float32)), tf.float32))))

In [12]:
x = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]))
y = tf.placeholder(dtype=tf.float32, shape=(None, Y_train.shape[1]))

loss_func = loss(x, y)
train_op = train(loss_func)


print("Run for Batch Size = 1")
mini_batch(1)

print("Run for Batch Size = 5")
mini_batch(5)

print("Run for Batch Size = 10")
mini_batch(10)

print("Run for Batch Size = 100")
mini_batch(100)



Run for Batch Size = 1
Accuracy : 
0.78089887
Run for Batch Size = 5
Accuracy : 
0.48876405
Run for Batch Size = 10
Accuracy : 
0.5224719
Run for Batch Size = 100
Accuracy : 
0.64606744
